### Import Block

In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

### Get Data

In [2]:
# Load shapefile.
state_shapes = gpd.read_file('States_Amazon_Biome/states_amazon_biome.shp')

In [3]:
%%time
# Load VIIRS data back into memory from CSV.
px_combined = pd.read_csv('pixel_data_filtered.csv')

Wall time: 16.5 s


In [4]:
%%time
# Convert to geopandas format.
px_combined = gpd.GeoDataFrame(px_combined, geometry = gpd.points_from_xy(px_combined['Lon'], px_combined['Lat']))

Wall time: 3min 8s


### Geographic Filtering

In [5]:
# Extract shape of Para
para_shape = state_shapes.iloc[4]['geometry']
para_shape

In [6]:
%%time
# Check pixels from dataframe to see if they're in Para.
px_combined['In_Para'] = px_combined['geometry'].within(para_shape, align=True)

Wall time: 3h 48min 20s


### Quality Check and Save

In [9]:
# Perform filtering.
px_para = px_combined[px_combined['In_Para'] == True]

In [11]:
# Check residual dataset percentage.
len(px_para) / len(px_combined)

0.11154101945684472

In [12]:
# Save filtered GeoDataFrame.
px_para.to_csv('VIIRS_Para.csv')

In [35]:
# Save without geotags.
px_para.drop('geometry',axis=1).to_csv('VIIRS_Para_No_Geo.csv')

### FIltering to September

In [34]:
# Dropping unused columns.
px_para = px_para.drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)

In [43]:
# Add date.
px_para['Date'] = pd.to_datetime(px_para['YYYYMMDD'].astype(str))

In [52]:
# Add month and year.
px_para['Month'] = pd.DatetimeIndex(px_para['Date']).month
px_para['Year'] = pd.DatetimeIndex(px_para['Date']).year

In [55]:
# Filter to September and see how much data we're working with.
px_para[px_para['Month'] == 9].Year.value_counts()

2017    128796
2020     99778
2019     38048
2018     31036
2021     25583
Name: Year, dtype: int64

In [56]:
# Save Septmber data.
px_para[px_para['Month'] == 9].to_csv('VIIRS_Para_September.csv')